In [ ]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

In [ ]:
# set up a desogn of a given dimension. Dimensions will be respected
# in the design rendering. Note the chip design in centred at origin (0,0)

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

#Reference to Ansys hfss QRenderer
hfss = design.renderers.hfss

gui = MetalGUI(design)

In [ ]:
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

In [ ]:
#Add 2 transmons to the design
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um',
    pocket_height = '650 um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        a = dict(loc_W=+1,loc_H=+1),
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        d = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 2 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+1.4mm', pos_y='0mm', orientation = '90', **options))
q2 = TransmonPocket(design, 'Q2', options=dict(
    pos_x='-0.6mm', pos_y='0mm', orientation = '90', **options))

gui.rebuild()
gui.autoscale()

In [ ]:
# Add 2 hangers consisting of capacitively coupled transmission lines
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='1mm',
                                                pos_y='3mm',
                                                coupling_length='200um'))
TQ2 = CoupledLineTee(design, 'TQ2', options=dict(pos_x='-1mm',
                                                pos_y='3mm',
                                                coupling_length='200um'))
gui.rebuild()
gui.autoscale()

In [ ]:
#Add 2 meandered CPWs connecting the transmons to the hangers.

ops=dict(fillet='90um')
design.overwrite_enabled = True

options1 = Dict(
    total_length='8mm',
    hfss_wire_bonds = True,
    pin_inputs=Dict(
        start_pin=Dict(
            component='TQ1',
            pin='second_end'),
        end_pin=Dict(
            component='Q1',
            pin='a')),
    lead=Dict(
        start_straight='0.1mm'),
    **ops
)

options2 = Dict(
    total_length='9mm',
    hfss_wire_bonds = True,
    pin_inputs=Dict(
        start_pin=Dict(
            component='TQ2',
            pin='second_end'),
        end_pin=Dict(
            component='Q2',
            pin= 'a')),
    lead=Dict(
        start_straight='0.1mm'),
    **ops
)

meanderQ1 = RouteMeander(design, 'meanderQ1', options=options1)
meanderQ2 = RouteMeander(design, 'meanderQ2', options=options2)

gui.rebuild()
gui.autoscale()

In [ ]:
# Add 2 open to grounds at the ends of the horizontal CPW
otg1 = OpenToGround(design, 'otg1', options = dict(pos_x='3mm',
                                                   pos_y='3mm'))
otg2 = OpenToGround(design, 'otg2', options = dict(pos_x = '-3mm',
                                                   pos_y='3mm',
                                                   orientation='180'))

gui.rebuild()
gui.autoscale()

In [ ]:
# Add 3 straight CPWs that comprise the long horizontal CPW.

ops_oR = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                  start_pin=Dict(
                      component='TQ1',
                      pin='prime_end'),
                  end_pin=Dict(
                      component='otg1',
                      pin='open')))
ops_mid = Dict(hfss_wire_bonds = True,
               pin_inputs=Dict(
                 start_pin=Dict(
                     component='TQ1',
                     pin='prime_start'),
                 end_pin=Dict(
                     component='TQ2',
                     pin='prime_end')))
ops_oL = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                  start_pin=Dict(
                      component='TQ2',
                      pin='prime_start'),
                  end_pin=Dict(
                      component='otg2',
                      pin='open')))

cpw_openRight = RouteStraight(design, 'cpw_openRight', options=ops_oR)
cpw_middle = RouteStraight(design, 'cpw_middle', options=ops_mid)
cpw_openLeft = RouteStraight(design, 'cpw_openLeft', options=ops_oL)

gui.rebuild()
gui.autoscale()


In [ ]:
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')


In [ ]:
hfss.connect_ansys()

In [ ]:
hfss.activate_drivenmodal_design("HangingResonators")

In [ ]:
# Set the buffer width at the edge of the design to be -.5 mm in 
# both directions.

hfss.options['x_buffer_width_mm'] = 0.5
hfss.options['y_buffer_width_mm'] = 0.5

In [ ]:
# Here, pin cpw_openRight_end and cpw_openLeft_end are converted
# into lumped ports, each with an impedance of 50 Ohms. Neither of the 
# junctions in Q1 or Q2 are rendered. As a reminder, arguments are given
# as First parameter: List of components to render (empty list if rendering
# whole Metal design) Second parameter: list of pins (qcomp, pin) with open
# endcaps Third parameter: List of pins (qcomp, pin, impedance) to render
# as lumped ports Fourth parameter: List of junctions (qcomp, qgeometry_name,
# impendance, draw_ind) to render as lumped ports or as lumped port in parallel
# with a sheet inductance Fifth parameter: List of junctions (qcomp, qgeometry_name)
# to omit altogether during rendering Sixth parameter: Whether to render chip
# via box plus buffer or fixed chip size

hfss.render_design([],
            [],
            [('cpw_openRight', 'end', 50), ('cpw_openLeft', 'end', 50)],
            [],
            [('Q1', 'rect_jj'), ('Q2', 'rect_jj')],
            box_plus_buffer=False)

In [ ]:
hfss.add_sweep(setup_name="Setup",
               name="Sweep",
               start_ghz=4.0,
               stop_ghz=8.0,
               count=4001,
               type="Interpolating")

In [ ]:
hfss.analyze_sweep('Sweep', 'Setup')

In [ ]:
hfss.plot_params(['S11', 'S21'])

In [ ]:
hfss.disconnect_ansys()